<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/Proj)Chatbot_for_part_time_worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치

In [ ]:
!pip install -U langchain openai langchain-google-genai

In [ ]:
!pip install PyMuPDF

In [ ]:
# import lib
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel

## Google Drive

In [ ]:
# google drive
from google.colab import drive
drive.mount("/content/drive/")

## API link

In [ ]:
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()   # https://makersuite.google.com/app/apikey

## PDF 읽기

In [ ]:
import fitz

path = "/content/drive/MyDrive/pdf/part_time_workers/"

doc_list = ["근로기준법", "산업재해보상보험법", "최저임금법", "헌법제32조"]
docs = {"근로기준법": "", "산업재해보상보험법": "", "최저임금법": "", "헌법제32조": ""}

for filename in os.listdir(path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(path, filename)
        filename = os.path.splitext(filename)[0]

        print(filename)
        pdf = fitz.open(pdf_path)
        for page in pdf:
          page = page.get_text()
          print(page)
          docs[filename] += page
        pdf.close()

In [ ]:
print(docs["근로기준법"])

In [ ]:
def create_chain(llm, template_prompt, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=template_prompt
        ),
        output_key=output_key,
        verbose=True,
    )


llm = ChatGoogleGenerativeAI(model="gemini-pro")

rule_1 = create_chain(
    llm=llm,
    template_prompt=docs["근로기준법"],
    output_key="text",
)
rule_2 = create_chain(
    llm=llm,
    template_prompt=docs["산업재해보상보험법"],
    output_key="text",
)
rule_3 = create_chain(
    llm=llm,
    template_prompt=docs["최저임금법"],
    output_key="text",
)
rule_4 = create_chain(
    llm=llm,
    template_prompt=docs["헌법제32조"],
    output_key="text",
)


destinations = [
    "산업재해보상보험법: 이 법의 목적에 적합할 경우에 이 키워드를 선택해줘. 목적: 이 법은 산업재해보상보험 사업을 시행하여 근로자의 업무상의 재해를 신속하고 공정하게 보상하며, 재해근로자의 재활 및 사회 복귀를 촉진하기 위하여 이에 필요한 보험시설을 설치ㆍ운영하고, 재해 예방과 그 밖에 근로자의 복지 증진을 위한 사업을 시행하여 근로자 보호에 이바지하는 것을 목적으로 한다.",
    "최저임금법: 이 법의 목적에 적합할 경우에 이 키워드를 선택해줘. 목적: 이 법은 근로자에 대하여 임금의 최저수준을 보장하여 근로자의 생활안정과 노동력의 질적 향상을 꾀함으로써 국민경제의 건전한 발전에 이바지하는 것을 목적으로 한다."
    "근로기준법: 이 법의 목적에 적합할 경우에 이 키워드를 선택해줘. 목적: 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는 국민경제의 발전을 꾀하는 것을 목적으로 한다.",
    "헌법제32조: 여성이 근로에서 받는 불이익에 관한 이야기는 이 키워드를 다른 키워드와 함께 선택해줘"
]
destinations = "\n".join(destinations)
router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate.from_template(
    template=router_prompt_template, output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)

multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains={
        "근로기준법": rule_1,
        "산업재해보상보험법": rule_2,
        "최저임금법": rule_3,
        "헌법제32조": rule_4,
    },
    default_chain=ConversationChain(llm=llm, output_key="text"),
)


class UserRequest(BaseModel):
    user_message: str


def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.dict()
    context["input"] = context["user_message"]
    answer = multi_prompt_chain.invoke(context)

    return {"answer": answer}

In [ ]:
user_data = {
    "user_message": "주휴수당을 받으려면 1주일에 몇시간 이상 일해야 해?"
}

In [ ]:
request_instance = UserRequest(**user_data)
gernerate_answer(request_instance)